# Purpose

1. 흐릿한 점을 주변의 명확한 사진(노드)로 추론해보는것, 소프트맥스로 나온 결과가 애매한 것들
2. 노드로 인식하고 극중 상황을 추론하는 (싸움/긴장 등)

# Import

In [3]:
import skimage
import numpy as np
from PIL import Image
import requests
import torch
from transformers import pipeline

In [4]:
import numpy as np
import os
from PIL import Image, ImageDraw
from datasets import load_dataset

# graph

In [5]:
cppe5 = load_dataset("cppe-5")

In [12]:
data_dict = {}
for z in range(len(cppe5['train'])) :
    image = cppe5["train"][z]["image"]
    annotations = cppe5["train"][z]["objects"]
    draw = ImageDraw.Draw(image)

    categories = cppe5["train"].features["objects"].feature["category"].names

    id2label = {index: x for index, x in enumerate(categories, start=0)}
    label2id = {v: k for k, v in id2label.items()}
    
    # for i in range(len(annotations["id"])):
    #     box = annotations["bbox"][i - 1]
    #     class_idx = annotations["category"][i - 1]
    #     x, y, w, h = tuple(box)
    #     draw.rectangle((x, y, x + w, y + h), outline="red", width=1)
    #     draw.text((x, y), id2label[class_idx], fill="white")

#     edge_matrix = np.zeros((len(categories), len(categories)), dtype=int)

#     for i in range(len(annotations['category'])-1):
#         edge_matrix[annotations['category'][i],annotations['category'][i+1]] = 1

#     edge_list = []

    # for i in range(edge_matrix.shape[0]):
    #     for j in range(edge_matrix.shape[1]):
    #         if edge_matrix[i, j] == 1:
    #             edge_list.append([i, j])
                
    data_dict[z] = {
        "image": image,
        "annotations": annotations,
        # "edge_matrix": edge_matrix,
        # "edge_list": edge_list
    }

In [13]:
data_dict[0]

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=943x663>,
 'annotations': {'id': [114, 115, 116, 117],
  'area': [3796, 1596, 152768, 81002],
  'bbox': [[302.0, 109.0, 73.0, 52.0],
   [810.0, 100.0, 57.0, 28.0],
   [160.0, 31.0, 248.0, 616.0],
   [741.0, 68.0, 202.0, 401.0]],
  'category': [4, 4, 0, 0]}}